#### Currently you can monitor the feed at this address:

http://10.0.0.201:7777


In [ ]:
#!/usr/bin/env python3

import io
import os
import sys
import time
import glob
import shutil
import picamera
import logging
import webbrowser 
import numpy as np
import smtplib, ssl
from numpy import asarray
import socketserver
from time import sleep
from threading import Condition
from http import server
from PIL import Image
from urllib.parse import parse_qs


notificationType = 0
captureStillOn = True

notDelay = 300 # 5 Minute
#notDelay = 900 # 15 Minute
#notDelay  = 3600 # 1 Hour
ownerEmail = "notify.lockindustries@gmail.com" #"michael.c.locker@gmail.com"
notificationAuth = "NotifyMe1!"

captureRate = 24 # yields one every Second (because it matches the framerate)
#captureRate = 240 # yields one every 10 Seconds
#captureRate = 1800 # yields one every Minute
#captureRate = 108000 # yields one every Hour
#captureRate = 2592000 # yields one every Day

shutdownFlag = 'R'
PAGE=str(open("index.html", "r").read())
styleSheet=str(open("style.css", "r").read())

count = 0
global imgNum
imgNum = 0
notTimerSet = time.perf_counter()
inferIdentity=False

def turingCheck(sample):
    # Provides space for human check
    sizedSampleArray = np.array(Image.open(io.BytesIO(sample)).resize((224,224),))
    result = False
    return result

def indentityCheck(sample):
    # Provides space for indetity check
    result = False
    return result

def engageBolt():
    # Provides space for engaging lock
    callback = True
    return callback

def disengageBolt():
    # Provides space for disengaging lock
    callback = True
    return callback

def logVisitor(inferenceValue=0):
    if infernceValue!=0:
        if infernceValue==1: # this should be a dict lookup
            inferenceName='myHomiesName'

    else:
        inferenceName='Unidentified Individual'
        

    # write a log entry concerning person
    return inferenceName


def extractFiles(folderPath, folderName, subFolder=''):
    
    # Setup path variables
    copyName = folderName+'_Extractable_'+(time.strftime("%m%d%y_%H%M%S", time.gmtime()))
    copyPath = folderPath+copyName
    zipPath = folderPath+subFolder+copyName
    transferredFiles = glob.glob(folderPath+folderName+'/*')
    
    # Copy files from src to destination
    shutil.copytree(folderName, copyPath)
    
    # Zip temporary files into target location
    try:
        shutil.make_archive(zipPath, 'zip', folderPath, copyName)
    except OSError:
        pass
    
    # Clean up transferred Images
    for f in transferredFiles:
        os.remove(f)
        
    # Clean up tempory file
    shutil.rmtree(copyPath)
    
    return


        
def mse(imageA, imageB):
    # Mean Squared Error sum of the squared 
    # difference will help determine change
    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1])
    return err

def sequenceCapture(frame):
    global count
    global imgNum
    global referenceImg1
    global referenceImg2
    global testImg
    global notTimerSet
    extractionThreshold = 500 #00
    count += 1
    visitorID = ''
    
    if count%captureRate==0:
        if count==captureRate:
            referenceImg1 = np.array(Image.open(io.BytesIO(frame)))
            referenceImg2 = np.array(Image.open(io.BytesIO(frame)))
        else:
            testImg = np.array(Image.open(io.BytesIO(frame)))
            imgMSE1 = int(mse(referenceImg1, testImg))
            imgMSE2 = int(mse(referenceImg2, testImg))
            referenceImg2 = referenceImg1
            referenceImg1 = testImg
            if (imgMSE1>=1000) or (imgMSE2>=1000):
                if captureStillOn==True:
                    if imgNum>=extractionThreshold:
                        extractFiles('/home/pi/CameraProject/','sampleImages','extraction/')
                        imgNum = 0
                    with open("sampleImages/img" + time.strftime("%m%d%y_%H%M%S", time.gmtime()) + ".jpg", "wb") as img:                        
                        img.write(frame)
                        print("Frame",imgNum,"Captured")
                        imgNum  += 1
                        
                if inferIdentity==True:
                    if turingCheck(frame):
                        if indentityCheck(frame):
                            disengageBolt()
                            visitorID = logVisitor()
                            sleep(30)
                            engageBolt()

                if notificationType==1 or notificationType==2:
                    notTimerComp = time.perf_counter()
                    if notTimerComp-notTimerSet >= notDelay:
                        print("Sending Notification")
                        notifyOwner(notificationType, visitorID) #,identityCheck.id)                             ### Create typed notifications
                        notTimerSet = notTimerComp



def handleInput(fields):
    print("Fields:", fields)
    test = str(fields.get('notAdd'))
    print("FieldsValue: ", test)
    if test=="['michael.c.locker@gmail.com']":
        global ownerEmail
        ownerEmail = 'michael.c.locker@gmail.com'
        print("Notification Email Changed to: 'michael.c.locker@gmail.com' ")
    elif test=="['jamedaful@gmail.com']":
        ownerEmail = 'jamedaful@gmail.com'
        print("Notification Email Changed to: 'jamedaful@gmail.com' ")
    elif test=="['notify.lockindustries@gmail.com']":
        ownerEmail = 'notify.lockindustries@gmail.com'
        print("Notification Email Changed to: 'notify.lockindustries@gmail.com' ")
    elif test=="['NotificationMotion']":
        global notificationType
        notificationType = 1
        print("Notifications are now set to motion. ")
    elif test=="['NotificationTuring']":
        notificationType = 2
        print("Notifications are now set to turing. ")
    elif test=="['NotificationOff']":
        notificationType = 0
        print("Notifications are now turned off. ")
    elif test=="['CaptureOn']":
        global captureStillOn
        captureStillOn = True
        print("Still Capturing is now turned on. ")
    elif test=="['CaptureOff']":
        captureStillOn = False
        print("Still Capturing is now turned off. ")
    elif test=="['ExtractImages']":
        extractFiles('/home/pi/CameraProject/','sampleImages','extraction/')
        global imgNum
        imgNum = 0
        print("Extracting Still Images. ") 
    elif test=="['ServerShutdown']":
        global shutdownFlag
        shutdownFlag = 'x'
        print("Server will shutdown")
    else:
        print("Notification Email Not Changed. Invalid Input")
    return


def notifyOwner(notificationSelection, notificationContent=""):
    port = 587  # For starttls
    smtp_server = "smtp.gmail.com"
    sender_email = "notify.lockindustries@gmail.com"
    receiver_email = ownerEmail
    #password = input("Type your password and press enter:")
    password = notificationAuth
    message = """\
    Subject: LockCam Notification
    
    """
    if notificationSelection==1 or notificationSelection==2:
        message += """\
        Motion has been detected.

        """
        if notificationContent!='':
            message+= """\
            The subject has been identified as """+notificationContent
        else:
            message+= """\
            The subject cannot be identified. """+notificationContent

    context = ssl.create_default_context()
    with smtplib.SMTP(smtp_server, port) as server:
        server.ehlo()  # Can be omitted
        server.starttls(context=context)
        server.ehlo()  # Can be omitted
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, message)


class StreamingOutput(object):

    def __init__(self):
        self.frame = None
        self.buffer = io.BytesIO()
        self.condition = Condition()

    def write(self, buf):
        if buf.startswith(b'\xff\xd8'):
            # New frame, copy the existing buffer's content and notify all
            # clients it's available
            self.buffer.truncate()
            with self.condition:
                self.frame = self.buffer.getvalue()
                self.condition.notify_all()
                sequenceCapture(self.frame)
#                 global imgNum
#                 camera.annotate_text_size = 6
#                 camera.annotate_foreground = picamera.Color('white') # white is also the default
#                 camera.annotate_foreground = picamera.Color(y=0.4, u=0, v=0) # dark
#                 camera.annotate_foreground = picamera.Color(y=0.2, u=0, v=0) # darker
#                 camera.annotate_foreground = picamera.Color(y=0.0, u=0, v=0) # average?
#                 camera.annotate_foreground = picamera.Color('black') # black
#                 camera.annotate_text = ("Captured "+str(imgNum)+" images")
#                 camera.annotate_text = str(imgNum)
                self.buffer.seek(0)

        return self.buffer.write(buf)

    
    
class StreamingHandler(server.BaseHTTPRequestHandler):
    
    def _set_headers(self):
        self.send_response(200)
        self.send_header('Content-type', 'text/html')
        self.end_headers()
    
    def do_POST(self):
        print(self.headers)

        content_length = int(self.headers.get('Content-Length', 0))
        content_string = self.rfile.read(content_length).decode("UTF-8")
        fields = parse_qs(content_string)
        handleInput(fields)
        
        self.send_response(302)
        self.send_header('Location', '/index.html')
        self.end_headers()
        self._set_headers()   
        return
    
    def do_GET(self):
        if self.path == '/':
            self.send_response(301)
            self.send_header('Location', '/index.html')
            self.end_headers()
        elif self.path == '/index.html':
            content = PAGE.encode('utf-8')
            self.send_response(200)
            self.send_header('Content-Type', 'text/html')
            self.send_header('Content-Length', len(content))
            self.end_headers()
            self.wfile.write(content)
        elif self.path == '/style.css':
            content = styleSheet.encode('utf-8')
            self.send_response(200)
            self.send_header('Content-Type', 'text/css')
            self.send_header('Content-Length', len(content))
            self.end_headers()
            self.wfile.write(content)
#         elif self.path == '/javascript.js':
#             content = styleSheet.encode('utf-8')
#             self.send_response(200)
#             self.send_header('Content-Type', 'text/javascript')
#             self.send_header('Content-Length', len(content))
#             self.end_headers()
#             self.wfile.write(content)
        elif self.path == '/stream.mjpg':
            self.send_response(200)
            self.send_header('Age', 0)
            self.send_header('Cache-Control', 'no-cache, private')
            self.send_header('Pragma', 'no-cache')
            self.send_header('Content-Type', 'multipart/x-mixed-replace; boundary=FRAME')
            self.end_headers()

            try:
                while True:
                    with output.condition:
                        output.condition.wait()
                        frame = output.frame
                        
                        self.wfile.write(b'--FRAME\r\n')
                        self.send_header('Content-Type', 'image/jpeg')
                        self.send_header('Content-Length', len(frame))
                        self.end_headers()
                        self.wfile.write(frame)
                        self.wfile.write(b'\r\n')

            except Exception as e:
                if shutdownFlag=="x":
                    camera.close()
                    server.shutdown()
                    server.server_close()
                    print("Remote Shutdown Requested.")
                logging.warning(
                    'Removed streaming client %s: %s',
                    self.client_address, str(e))
        else:
            self.send_error(404)
            self.end_headers()

            
            
class StreamingServer(socketserver.ThreadingMixIn, server.HTTPServer):
    allow_reuse_address = True
    daemon_threads = True
    global imgNum

with picamera.PiCamera(resolution='240x240', framerate=24) as camera: #'729x729', framerate=6) as camera:
    camera.rotation = 270
    output = StreamingOutput()
    camera.start_recording(output, format='mjpeg')
#    webbrowser.open('http://10.0.0.201:7777', new=0, autoraise=True)
        
    try:
        address = ('', 7777)
        server = StreamingServer(address, StreamingHandler)
        print("Starting Server . . .")
        if notificationType==0:
            print("Notifications are disabled.")
        elif notificationType==1:
            print("Notifications are set to Motion.")
        elif notificationType==2:
            print("Notifications are set to Turing.")
        server.serve_forever() #poll_interval=0.5)
        
    except KeyboardInterrupt:
        camera.close()
        server.shutdown() # Stop the serve_forever
        server.server_close()  # Close also the socket.
        print("Shutdown requested")
        
    except Exception:
        print("Encountered Unhandled Error!")
        camera.close()
        server.shutdown() # Stop the serve_forever
        server.server_close()  # Close also the socket.

    finally:
        camera.close()
        server.shutdown() # Stop the serve_forever
        server.server_close()  # Close also the socket.


Starting Server . . .
Notifications are disabled.


10.0.0.163 - - [27/Oct/2020 14:30:06] "GET /index.html HTTP/1.1" 200 -
10.0.0.163 - - [27/Oct/2020 14:30:06] "GET /style.css HTTP/1.1" 200 -
10.0.0.163 - - [27/Oct/2020 14:30:06] "GET /stream.mjpg HTTP/1.1" 200 -
10.0.0.163 - - [27/Oct/2020 14:30:06] code 404, message Not Found
10.0.0.163 - - [27/Oct/2020 14:30:06] "GET /favicon.ico HTTP/1.1" 404 -


Frame 0 Captured
Frame 1 Captured
Frame 2 Captured
Frame 3 Captured
Frame 4 Captured
Frame 5 Captured
Frame 6 Captured
Frame 7 Captured
Frame 8 Captured
Frame 9 Captured
Frame 10 Captured
Frame 11 Captured
Frame 12 Captured
Frame 13 Captured
Frame 14 Captured
Frame 15 Captured
Frame 16 Captured
Frame 17 Captured
Frame 18 Captured
Frame 19 Captured
Frame 20 Captured
Frame 21 Captured
Frame 22 Captured
Frame 23 Captured
Frame 24 Captured
Frame 25 Captured
Frame 26 Captured
Frame 27 Captured
Frame 28 Captured
Frame 29 Captured
Frame 30 Captured
Frame 31 Captured
Frame 32 Captured
Frame 33 Captured
Frame 34 Captured
Frame 35 Captured
Frame 36 Captured
Frame 37 Captured
Frame 38 Captured
Frame 39 Captured
Frame 40 Captured
Frame 41 Captured
Frame 42 Captured
Frame 43 Captured
Frame 44 Captured
Frame 45 Captured
Frame 46 Captured
Frame 47 Captured
Frame 48 Captured
Frame 49 Captured
Frame 50 Captured
Frame 51 Captured
Frame 52 Captured
Frame 53 Captured
Frame 54 Captured
Frame 55 Captured
Fr

10.0.0.163 - - [27/Oct/2020 16:16:29] "GET /index.html HTTP/1.1" 200 -
10.0.0.163 - - [27/Oct/2020 16:16:30] "GET /style.css HTTP/1.1" 200 -
10.0.0.163 - - [27/Oct/2020 16:16:30] "GET /stream.mjpg HTTP/1.1" 200 -
10.0.0.163 - - [27/Oct/2020 16:16:30] code 404, message Not Found
10.0.0.163 - - [27/Oct/2020 16:16:30] "GET /favicon.ico HTTP/1.1" 404 -
10.0.0.163 - - [27/Oct/2020 17:05:57] "GET /index.html HTTP/1.1" 200 -
10.0.0.163 - - [27/Oct/2020 17:05:57] "GET /style.css HTTP/1.1" 200 -
10.0.0.163 - - [27/Oct/2020 17:05:57] "GET /stream.mjpg HTTP/1.1" 200 -
10.0.0.163 - - [27/Oct/2020 17:05:57] code 404, message Not Found
10.0.0.163 - - [27/Oct/2020 17:05:57] "GET /favicon.ico HTTP/1.1" 404 -


Frame 187 Captured
Frame 188 Captured
Frame 189 Captured
Frame 190 Captured
Frame 191 Captured
Frame 192 Captured
Frame 193 Captured
Frame 194 Captured
Frame 195 Captured
Frame 196 Captured
Frame 197 Captured
Frame 198 Captured
Frame 199 Captured
Frame 200 Captured
Frame 201 Captured
Frame 202 Captured
Frame 203 Captured
Frame 204 Captured
Frame 205 Captured
Frame 206 Captured
Frame 207 Captured
Frame 208 Captured
Frame 209 Captured
Frame 210 Captured
Frame 211 Captured
Frame 212 Captured
Frame 213 Captured
Frame 214 Captured
Frame 215 Captured
Frame 216 Captured
Frame 217 Captured


...

Frame 218 Captured
Frame 219 Captured
Frame 220 Captured
Frame 221 Captured
Frame 222 Captured
Frame 223 Captured
Frame 224 Captured
Frame 225 Captured
Frame 226 Captured
Frame 227 Captured
Frame 228 Captured
Frame 229 Captured
Frame 230 Captured
Frame 231 Captured
Frame 232 Captured
Frame 233 Captured
Frame 234 Captured
Frame 235 Captured
Frame 236 Captured
Frame 237 Captured
Frame 238 Captured
Frame 239 Captured
Frame 240 Captured
Frame 241 Captured
Frame 242 Captured
Frame 243 Captured
Frame 244 Captured
Frame 245 Captured
Frame 246 Captured
Frame 247 Captured
Frame 248 Captured
Frame 249 Captured
Frame 250 Captured
Frame 251 Captured
Frame 252 Captured
Frame 253 Captured
Frame 254 Captured
Frame 255 Captured
Frame 256 Captured
Frame 257 Captured
Frame 258 Captured
Frame 259 Captured
Frame 260 Captured
Frame 261 Captured
Frame 262 Captured
Frame 263 Captured
Frame 264 Captured
Frame 265 Captured
Frame 266 Captured
Frame 267 Captured
Frame 268 Captured
Frame 269 Captured
Frame 270 Ca

10.0.0.163 - - [27/Oct/2020 18:47:49] "GET /index.html HTTP/1.1" 200 -
10.0.0.163 - - [27/Oct/2020 18:47:49] "GET /style.css HTTP/1.1" 200 -
10.0.0.163 - - [27/Oct/2020 18:47:49] "GET /stream.mjpg HTTP/1.1" 200 -
10.0.0.163 - - [27/Oct/2020 18:47:49] code 404, message Not Found
10.0.0.163 - - [27/Oct/2020 18:47:49] "GET /favicon.ico HTTP/1.1" 404 -


Frame 322 Captured
Frame 323 Captured
Frame 324 Captured
Frame 325 Captured
Frame 326 Captured
Frame 327 Captured
Frame 328 Captured
Frame 329 Captured
Frame 330 Captured
Frame 331 Captured
Frame 332 Captured
Frame 333 Captured
Frame 334 Captured
Frame 335 Captured
Frame 336 Captured
Frame 337 Captured
Frame 338 Captured
Frame 339 Captured
Frame 340 Captured
Frame 341 Captured
Frame 342 Captured
Frame 343 Captured
Frame 344 Captured
Frame 345 Captured
Frame 346 Captured
Frame 347 Captured
Frame 348 Captured
Frame 349 Captured
Frame 350 Captured
Frame 351 Captured
Frame 352 Captured
Frame 353 Captured
Frame 354 Captured
Frame 355 Captured
Frame 356 Captured
Frame 357 Captured
Frame 358 Captured
Frame 359 Captured
Frame 360 Captured
Frame 361 Captured
Frame 362 Captured
Frame 363 Captured
Frame 364 Captured
Frame 365 Captured
Frame 366 Captured
Frame 367 Captured
Frame 368 Captured
Frame 369 Captured
Frame 370 Captured
Frame 371 Captured
Frame 372 Captured
Frame 373 Captured
Frame 374 Ca

10.0.0.163 - - [27/Oct/2020 23:12:03] "GET /index.html HTTP/1.1" 200 -
10.0.0.163 - - [27/Oct/2020 23:12:03] "GET /style.css HTTP/1.1" 200 -
10.0.0.163 - - [27/Oct/2020 23:12:03] "GET /stream.mjpg HTTP/1.1" 200 -
10.0.0.163 - - [27/Oct/2020 23:12:03] code 404, message Not Found
10.0.0.163 - - [27/Oct/2020 23:12:03] "GET /favicon.ico HTTP/1.1" 404 -


Frame 223 Captured
Frame 224 Captured
Frame 225 Captured
Frame 226 Captured


#### Currently you can monitor the feed at this address:

http://10.0.0.201:7777


In [5]:
# The Sample Extractor

import os
import time
import shutil
import glob

def extractFiles(folderPath, folderName, subFolder=''):
    
    # Setup path variables
    copyName = folderName+'_Extractable_'+(time.strftime("%m%d%y_%H%M%S", time.gmtime()))
    copyPath = folderPath+copyName
    zipPath = folderPath+subFolder+copyName
    transferredFiles = glob.glob(folderPath+folderName+'/*')
    
    # Copy files from src to destination
    shutil.copytree(folderName, copyPath)
    
    # Zip temporary files into target location
    try:
        shutil.make_archive(zipPath, 'zip', folderPath, copyName)
    except OSError:
        pass
    
    # Clean up transferred Images
    for f in transferredFiles:
        os.remove(f)
        
    # Clean up tempory file
    shutil.rmtree(copyPath)
    
    return


extractFiles('/home/pi/CameraProject/','sampleImages','extraction/')

In [ ]:
### This command works to copy zipped files from remote raspberry to local windows machine (RUN ON WINDOWS CMD LINE)
scp pi@10.0.0.201:/home/pi/CameraProject/extraction/*  C:/Users/micha/Desktop/UnsortedImages

In [ ]:
### This command works to copy all files from remote raspberry to local windows machine (RUN ON WINDOWS CMD LINE)
scp pi@10.0.0.201:/home/pi/CameraProject/sampleImages/*  C:/Users/micha/Desktop/UnsortedImages

In [ ]:
## Image Copier for making test backups
!cp -r /home/pi/CameraProject/sampleImagesBackup/* /home/pi/CameraProject/sampleImages/